Preparing to submit wold stranded samples....

This got a little messy because a couple libraries needed a top up.

In [1]:
import os
import sys
import requests
import pandas
import paramiko
import json
from IPython import display
from pathlib import Path

In [2]:
from curation_common import *
from htsworkflow.submission.encoded import DCCValidator

In [3]:
from htsworkflow.submission.encoded import Document
from htsworkflow.submission.aws_submission import run_aws_cp

In [4]:
# live server & control file
server = ENCODED('www.encodeproject.org')
spreadsheet_name = Path('~/woldlab/ENCODE/stranded-23204-23231-mouse-hippocampus.xlsx').expanduser()
engine=None
#engine='odf'

# test server & datafile
#server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-encode3-limb-2017-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [5]:
award = 'UM1HG009443'

# Lookup biosample ontologies

Lookup any biosample ontologies that are already present

In [6]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)

for i, row in biosample.iterrows():
    if not pandas.isnull(row.accession) and row.accession.startswith('E'):
        obj = server.get_json(row.accession)
        biosample_ontology = obj['biosample_ontology']
        if isinstance(biosample_ontology, dict):
            biosample.loc[i, 'biosample_ontology'] = biosample_ontology['@id']
            biosample.loc[i, 'biosample_term_name:skip'] = biosample_ontology['term_name']
            
biosample

,uuid,accession,aliases:array,biosample_ontology,biosample_term_name:skip,description,date_obtained:date,starting_amount:integer,starting_amount_units,model_organism_age,model_organism_age_units,model_organism_sex,library_id:skip,cDNA_sample:skip,organism,donor,source,lab,award
0,c3f57bda-022e-4bf0-9db6-c5ddfa59ef06,ENCBS739AAH,barbara-wold:ENC4_cDNA493,/biosample-types/tissue_UBERON_0002305/,layer of hippocampus,"hippocampus_B6Cast F1_PND10_3x female_4,5,6_EN...",2020-10-21,3,items,10,day,female,23204,ENC4_cDNA493,/organisms/mouse/,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443
1,c7751ccd-3a41-43e3-a572-75078bcb7bef,ENCBS286DXZ,barbara-wold:ENC4_cDNA494,/biosample-types/tissue_UBERON_0002305/,layer of hippocampus,"hippocampus_B6Cast F1_PND10_3x female_10,11,12...",2021-01-05,3,items,10,day,female,23205,ENC4_cDNA494,/organisms/mouse/,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443
2,673bffa4-8370-4d87-a2f8-52c7454fb89b,ENCBS435SED,barbara-wold:ENC4_cDNA495,/biosample-types/tissue_UBERON_0002305/,layer of hippocampus,"hippocampus_B6Cast F1_PND10_3x male_4,5,6_ENC4...",2020-10-21,3,items,10,day,male,23206,ENC4_cDNA495,/organisms/mouse/,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443
3,580b62f6-d1a1-47ae-abdc-b13c2438a269,ENCBS426ZTY,barbara-wold:ENC4_cDNA496,/biosample-types/tissue_UBERON_0002305/,layer of hippocampus,"hippocampus_B6Cast F1_PND10_3x male_16,17,18_E...",2021-01-26,3,items,10,day,male,23207,ENC4_cDNA496,/organisms/mouse/,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443
4,bfc2d5e0-794c-4be9-a41e-7150cb91148a,ENCBS229LCL,barbara-wold:ENC4_cDNA497,/biosample-types/tissue_UBERON_0002305/,layer of hippocampus,"hippocampus_B6CastF1_PND14_3x female_4,5,6_ENC...",2020-12-18,3,items,14,day,female,23208,ENC4_cDNA497,/organisms/mouse/,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443
5,7083b95a-01b7-417f-9f36-5e413442d36b,ENCBS708FHL,barbara-wold:ENC4_cDNA498,/biosample-types/tissue_UBERON_0002305/,layer of hippocampus,"hippocampus_B6CastF1_PND14_3x female_10,11,12_...",2021-01-08,3,items,14,day,female,23209,ENC4_cDNA498,/organisms/mouse/,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443
6,bfb0bcfc-182c-4396-b381-234f8e1090fa,ENCBS642ONZ,barbara-wold:ENC4_cDNA499,/biosample-types/tissue_UBERON_0002305/,layer of hippocampus,"hippocampus_B6CastF1_PND14_3x male_7,8,9_ENC4_...",2021-01-08,3,items,14,day,male,23210,ENC4_cDNA499,/organisms/mouse/,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443
7,8fb5b8ec-15f6-4a28-a212-48c284ff7af6,ENCBS299AJA,barbara-wold:ENC4_cDNA500,/biosample-types/tissue_UBERON_0002305/,layer of hippocampus,"hippocampus_B6CastF1_PND14_3x male_13,14,15_EN...",2021-03-02,3,items,14,day,male,23211,ENC4_cDNA500,/organisms/mouse/,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443
8,85f44888-0e87-49ab-8bfa-7af8d727e04b,ENCBS690ORU,barbara-wold:ENC4_cDNA501,/biosample-types/tissue_UBERON_0002305/,layer of hippocampus,hippocampus_B6CastF1_PND25_female_11_ENC4_cDNA501,2020-11-30,1,items,25,day,female,23212,ENC4_cDNA501,/organisms/mouse/,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443
9,811fd647-012c-429b-a5e1-6de895e602cf,ENCBS369QTC,barbara-wold:ENC4_cDNA502,/biosample-types/tissue_UBERON_0002305/,layer of hippocampus,hippocampus_B6CastF1_PND25_female_13_ENC4_cDNA502,2020-11-30,1,items,25,day,female,23213,ENC4_cDNA502,/organisms/mouse/,/mouse-donors/ENCDO509HIY/,/sources/jackson-labs/,barbara-wold,UM1HG009443


# Register Biosamples

In [7]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)
created = server.post_sheet('/biosamples/', biosample, 
                            verbose=True,
                            dry_run=True, 
                            validator=validator)
print(len(created))

0


In [8]:
if created:
    biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

## Forgot to post model_organism_sex

In [22]:
print(spreadsheet_name)
biosamples = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)
for i, row in biosamples.iterrows():
    biosample = server.get_json(row['accession'])
    model_organism_age = biosample.get('model_organism_sex')
    if model_organism_age != row['model_organism_sex']:
        print("{} is {} should be {}".format(row['accession'], model_organism_age, row['model_organism_sex']))
        #print(server.patch_json(
        #    '/biosamples/{}/'.format(row['accession']),
        #    {'model_organism_sex': row['model_organism_sex']}))

/home/diane/woldlab/ENCODE/stranded-23204-23231-mouse-hippocampus.xlsx


# Register Libraries

In [9]:
print(spreadsheet_name)
libraries = pandas.read_excel(spreadsheet_name, sheet_name='Library', header=0, engine=engine)
created = server.post_sheet('/libraries/', 
                            libraries, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

/home/diane/woldlab/ENCODE/stranded-23204-23231-mouse-hippocampus.xlsx
0


In [10]:
if created:
    libraries.to_excel('/dev/shm/libraries.xlsx', index=False)

# Register Experiments

In [11]:
print(server.server)
experiments = pandas.read_excel(spreadsheet_name, sheet_name='Experiment', header=0, engine=engine)
experiments = experiments[experiments['accession'] != 'barbara approval needed']
created = server.post_sheet('/experiments/', 
                            experiments, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

www.encodeproject.org
0


In [12]:
if created:
    experiments.to_excel('/dev/shm/experiments.xlsx', index=False)

In [13]:
#print(server.server)
#experiments = pandas.read_excel(spreadsheet_name, sheet_name='Experiment', header=0, engine=engine)
#for i, row in experiments.iterrows():
#    obj = server.get_json(row['accession'])
#    updates = {}
#    if obj['aliases'] != row['aliases:array']:
#        updates['aliases'] = [row['aliases:array']]
#    if obj['description'] != row['description']:
#        updates['description'] = row['description']

#    if len(updates) > 0:
#        print(obj['@id'], updates)
        #print(server.patch_json(obj['@id'], updates))

# Register Replicates

In [14]:
print(server.server)
print(spreadsheet_name)
replicates = pandas.read_excel(spreadsheet_name, sheet_name='Replicate', header=0, engine=engine)
replicates = replicates[replicates['uuid'] != 'barbara approval needed']
created = server.post_sheet('/replicates/',
                            replicates, 
                            verbose=True,
                            dry_run=True, 
                            validator=validator)
print(len(created))

www.encodeproject.org
/home/diane/woldlab/ENCODE/stranded-23204-23231-mouse-hippocampus.xlsx
0


In [15]:
if created:
    replicates.to_excel('/dev/shm/replicates.xlsx', index=False)

# Check Files

In [16]:
files = pandas.read_excel(spreadsheet_name, sheet_name='File', header=0, engine=engine)
created = server.post_sheet('/files/', files, verbose=True, dry_run=True, validator=validator)
print(len(created))

0
